# LMNA

In [ ]:
import genophenocorr
print(f"Using genophenocorr version {genophenocorr.__version__}")

In [ ]:
import hpotk

fpath_hpo = 'https://github.com/obophenotype/human-phenotype-ontology/releases/download/v2024-02-08/hp.json'
hpo = hpotk.load_minimal_ontology(fpath_hpo)

In [ ]:
from genophenocorr.preprocessing import configure_caching_cohort_creator, load_phenopacket_folder

fpath_phenopackets = 'input'
cohort_creator = configure_caching_cohort_creator(hpo)
cohort = load_phenopacket_folder(fpath_phenopackets, cohort_creator)

## Transcript
Get it by looking for gene symbol in https://www.ncbi.nlm.nih.gov/clinvar/ look for pathogenic
zcopy it and go to https://variantvalidator.org/service/validate/, then scroll down to copy GRCH38 string, copy this and put into same website

looking for MANE select plus clinical (most important transcript for clinic) if doesnt exist, use clinvar result

In [ ]:
tx_id = 'NM_170707.4'

## Explore Cohort

In [ ]:
from IPython.display import HTML, display
from genophenocorr.view import CohortViewer

viewer = CohortViewer(hpo)

In [ ]:
display(HTML(viewer.cohort_summary_table(cohort)))

In [ ]:
display(HTML(viewer.hpo_term_counts_table(cohort))) ## Add Labels to output

In [ ]:
display(HTML(viewer.variants_table(cohort, tx_id))) 

## Configure the analysis

In [ ]:
from genophenocorr.analysis import configure_cohort_analysis, CohortAnalysisConfiguration
from genophenocorr.analysis.predicate import BooleanPredicate, GroupingPredicate

analysis_config = CohortAnalysisConfiguration.builder() \
    .missing_implies_excluded(True) \
    .pval_correction('fdr_bh') \
    .min_perc_patients_w_hpo(0.1) \
    .build()
analysis = configure_cohort_analysis(cohort, hpo, analysis_config)

# did not work because we didn#'t get the right transcript
# frameshift = analysis.compare_by_variant_effect(VariantEffect.FRAMESHIFT_VARIANT, tx_id)
# frameshift.summarize(hpo, BooleanPredicate.YES)
var_single = analysis.compare_by_variant_key('1_156138613_156138613_C_T')
var_single.summarize(hpo, BooleanPredicate.YES)